In [29]:
import random
import importlib
import tictactoe
importlib.reload(tictactoe);
from tictactoe import TicTacToe


In [30]:
#
# Hyper parameters
# 
alpha = 0.2
gamma = 0.9
epsilon = 0.3

In [27]:
Q = {}
games = 0

In [32]:
#
#validation
#
def validate(iterations):
    wins = 0
    draws = 0
    losses = 0
    for _ in range(iterations):
        env = TicTacToe()
        qplayer = random.choice([env.player, env.opponent])
        done = False
        while not done:
            state = env.state
            if qplayer == env.player:
                action = max([a for a in range(9) if env.is_valid(a)], key = lambda x: Q.get((state, x), 0))
            else:
                action = random.choice([a for a in range(9) if env.is_valid(a)])
            env.move(action)
            if env.is_won():
                if qplayer == env.opponent:
                    wins += 1
                else:
                    losses += 1
                done = True
            elif env.is_full():
                draws += 1
                done = True

    print(f'Result: {100*(wins+draws)/iterations:.2f}% ({wins}/{draws}/{losses})')

In [ ]:
#
# TRAINING
#
log_interval = 100000
eval_interval = 200000
eval_iterations = 20000
lastLenQ = len(Q)
for _ in range(20000000):
    env = TicTacToe()
    done = False
    games += 1

    while not done:
        state = env.state
        
        if random.uniform(0, 1) < epsilon:
            action = random.choice([a for a in range(9) if env.is_valid(a)])
        else:
            action = max([a for a in range(9) if env.is_valid(a)], key=lambda x: Q.get((state, x), 0))
        
        env.move(action)

        if env.is_won():
            Q[(state, action)] = 1
            done = True
        elif env.is_full():
            Q[(state, action)] = 0
            done = True
        else:
            next_state = env.state
            next_max = -max([Q.get((next_state, a), 0) for a in range(9) if env.is_valid(a)])
            oldq = Q.get((state, action), None)
            targetq = -0.1 + gamma * next_max
            if oldq is None:
                Q[(state, action)] = targetq
            else:
                Q[(state, action)] = oldq + alpha*(targetq - oldq)
            
    if games % log_interval == 0:
        lenq = len(Q)
        print(f'{games}: Q has {lenq} (+{lenq - lastLenQ}) zeros: {sum([1 for q in Q.values() if q == 0])}')
        lastLenQ = lenq
    if games % eval_interval == 0:
        print(f'{games}: validating...')
        validate(eval_iterations)


In [21]:
env = TicTacToe()

In [ ]:
env.move(2)
env.render()

In [ ]:
state = env.state
action = max([a for a in range(9) if env.is_valid(a)], key=lambda x: Q.get((state, x), 0))
env.move(action)
print(action)
env.render()